# Preparations

---

## *Requirements*

*In order to run this notebook we rely on specifc package-versions; here we will downgrade these nessecarry packages.*

*After it's completion it is **required to restart** current running runtime-enviroment, this can be done by clicking 'Runtime' > 'Reset runtime' in the menubar.*

In [ ]:
#!pip2 uninstall -y numpy
#!pip3 uninstall -y numpy
#!pip2 install numpy==1.13.0
#!pip3 install numpy==1.13.0
#!pip uninstall -y torchtext fastai
#!pip install --upgrade numpy==1.13.0 
#!pip install torch==0.4.1 torchvision==0.1.9

## *Versioning*

In [ ]:
import torch
import PIL
import numpy as np

print('Installed Packegesversions:\n - PyTroch: {}\n - Pillow:  {}\n - Numpy:   {}'.format(torch.__version__, PIL.__version__, np.__version__))

## *Imports*

In [ ]:
import sys
import os
import shutil
from glob import glob

from torchvision import models
from PIL import Image

import math

from IPython.display import HTML, display
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


## *CNN-Visualization-Framework*

* *Author: Utku Ozbulak*

* *Year: 2019*

* *Github: [utkuozbulak/pytorch-cnn-visualizations](https://github.com/utkuozbulak/pytorch-cnn-visualizations)*

We start by cloning this repository and checkout at a specifc commit:

In [ ]:
%%capture
!git clone https://github.com/utkuozbulak/pytorch-cnn-visualizations.git
%cd /content/pytorch-cnn-visualizations
!git checkout 66af4935d76cb0b597c33068026ca2a8e1c562dc
%cd /content

Now will make the delcared classes, variables and functions importable:

In [ ]:
%%capture
%cd /content/pytorch-cnn-visualizations
with open('__init__.py', 'w') as f:
  f.write('')
%cd /content
sys.path.append('/content/pytorch-cnn-visualizations/src')

And will finally import used classes, variables and functions into this python-enviorment:

In [ ]:
from gradcam import GradCam, CamExtractor
from misc_functions import preprocess_image, save_image, apply_colormap_on_image

## *Utilities*

Here we declare some usefull functions. 

To show a HTML-progress bar when doing some intesive tasks:

In [ ]:
PROGRESS_BAR = None
PROGRESS_BAR_VALUE = 0
PROGRESS_BAR_MAX_VALUE = 100

def progressbar(value, max_value):
  """
  Creates a HTML-Progressbar

  @param value: the value to set the prograssbar to
  @param max_value: the max-value to set the prograssbar to
  """
  return HTML(f'<progress value=\'{value/max_value}\' max=\'{max}\' style=\'width: 100%\'>\'{value}%\'</progress><label>Finished {value} / {max_value}</label>')   

def display_progress_bar(init_value, max_value):
  """
  Initializes and displays a HTML-Progressbar

  @param init_value: the initial value to set the prograssbar to
  @param max_value: the max-value to set the prograssbar to
  """
  global PROGRESS_BAR
  global PROGRESS_BAR_VALUE
  global PROGRESS_BAR_MAX_VALUE
  PROGRESS_BAR_VALUE = init_value
  PROGRESS_BAR_MAX_VALUE = max_value
  PROGRESS_BAR = display(progressbar(PROGRESS_BAR_VALUE, PROGRESS_BAR_MAX_VALUE), display_id=True)

def update_progress_bar():
  """
  Will incrementaly update the progress-value and update the displayed HTML-Progressbar
  """
  global PROGRESS_BAR
  global PROGRESS_BAR_VALUE
  global PROGRESS_BAR_MAX_VALUE
  PROGRESS_BAR_VALUE = PROGRESS_BAR_VALUE + 1
  PROGRESS_BAR.update(progressbar(PROGRESS_BAR_VALUE, PROGRESS_BAR_MAX_VALUE))

Some functionality to display multiple images:

In [ ]:
%matplotlib inline

def extract_first_numbers(txt: str) -> str:
  """
  Will extract all numbers out of given string

  @param txt: the text to extract from

  @return: the extracted numbers 
  """
  return ''.join([s for s in txt if s.isdigit()])

def plot_images(image_paths: [str], columns: int, title: str, figsize: (int, int), tfunc):
  """
  Will plot given images

  @param image_paths: the paths of the images to display
  @param columns: number of colums to use
  @param title: the title of the figure
  @param figsize: the sizes used for the figure
  @param tfunc: function called to transform a filename into a more meaningfull str
  """
  rows = math.ceil(len(image_paths)/columns)
  fig, axes = plt.subplots(nrows=rows, ncols=columns, figsize=figsize)

  # column titles
  column_titles = []
  for path in image_paths[0:3]:
    column_titles.append(tfunc(path))
  for ax, ct in zip(axes[0], column_titles):
    ax.set_title(ct, fontsize='16')

  # row titles
  row_titles = []
  for image_path in image_paths[::columns]:
    row_titles.append('Layer ' + extract_first_numbers(image_path.split('/')[-1]))
  for ax, rt in zip(axes[:,0], row_titles):
    ax.set_ylabel(rt, rotation=90, fontsize='16')

  # image
  for i, cell in enumerate(axes.flat):
    cell.imshow(mpimg.imread(image_paths[i]))

  # styling
  fig.text(0.5, 1.003, title, fontsize='20', horizontalalignment='center', verticalalignment='top')
  fig.tight_layout()
  fig.patch.set_facecolor('xkcd:white')
  plt.show()

Other utility-functions

In [ ]:
def chunks(lst, n):
    """ 
    Yield successive n-sized chunks from lst.

    @param lst: the list to split
    @param n: the chunks to split the given list into
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Image-Samples

---

Here we declare a helper-class for our sample-images, which manages indices and offered some useful access methods to oure sample data:

In [ ]:
class ImageSamples:
  """
  A helper-class for Image-Sampls
  """
  
  def __init__(self, image_paths: [], targeted_class: [] = None, start_index: int = -1):
    """
    Init

    @param image_paths: list of paths to images
    @param targeted_class: list of targeted classes, if None the most likely detected class will be targeted
    @param start_index: internal inital index
    """
    self.__samples = image_paths
    assert len(self.__samples) >= 1, 'There must be at least one given sample'
    if targeted_class is None:
      self.__classes = [None] * len(self.__samples)
    else:
      self.__classes = targeted_class
      assert len(self.__samples) == len(self.__classes)  , 'Targeted classes must be of same length as given samples'
    self.__i = start_index
  
  def next(self) -> bool:
    """
    Will incrementality increase the index internaly used.

    @return: whether there is a element left
    """
    i = 1 + self.__i
    if i >= len(self.__samples) or i < 0:
      return False
    else:
      self.__i = i
      return True

  def reset_index(self):
    """
    Will reset the index, tho the next-function can be called again
    """
    self.__i = -1
  
  def get_preprocessed_image(self, colorcodec: str = 'RGB') -> Image:
    """
    Will return preprocess the current image for current index

    @param colorcodec: the colorcodec used to transform the image

    @return: the preprocessed image
    """
    return preprocess_image(Image.open(self.__samples[self.__i]).convert(colorcodec), resize_im=True)

  def get_original_image(self, colorcodec: str = 'RGB') -> Image:
    """
    Will return the original image for current index

    @param colorcodec: the colorcodec used to transform the image

    @return: the original image
    """
    return Image.open(self.__samples[self.__i]).convert(colorcodec)

  def get_file_name(self) -> str:
    """
    Will return the filename for current index

    @return: the filename (without file-extension)
    """
    return self.__samples[self.__i].split('/')[-1].split('.')[0]

  def get_sample(self) -> str:
    """
    Will return the set sample for current index

    @return: the sample 
    """
    return self.__samples[self.__i]

  def get_targeted_class(self) -> int:
    """
    Will return targeted class for current index

    @return: the number of targeted class
    """
    return self.__classes[self.__i]


  def __len__(self):
    return len(self.__samples)
  

In [ ]:
def default_image_samples() -> ImageSamples:
  """
  Will return a 'default' ImageSamples-object

  @return: a ImageSamples-object with some predefined values
  """
  return ImageSamples(image_paths = [
    '/content/pytorch-cnn-visualizations/input_images/snake.jpg',   
    '/content/pytorch-cnn-visualizations/input_images/cat_dog.png', 
  ])

# XAI-Functionality
---

## *GradientCam*

In [ ]:
class GradientCam:
  
  def __init__(self, gradient_cam_export_path: str = 'grad_cam', cam_heatmap_file_desc: str = '_cam_heatmap', cam_on_image_heatmap_file_desc: str = '_cam_on_image', cam_grayscale_file_desc: str = '_cam_grayscale', save_file_type: str = '.png'):
    self.grad_cam = None
    self.cam = None
    self.gradient_cam_export_path = gradient_cam_export_path
    self.cam_heatmap_file_desc = cam_heatmap_file_desc
    self.cam_on_image_heatmap_file_desc = cam_on_image_heatmap_file_desc
    self.cam_grayscale_file_desc = cam_grayscale_file_desc
    if not save_file_type[0] == '.':
      self.save_file_type = '.' + save_file_type
    else:
      self.save_file_type = save_file_type
  
  def run(self, model_wrapper, sample: ImageSamples, target_layer):
    #print(f'Calculating GradientCam for Image \'{sample.get_file_name()}\' ...')
    self.grad_cam = GradCam(model_wrapper.model, target_layer)
    self.cam = self.grad_cam.generate_cam(sample.get_preprocessed_image(), sample.get_targeted_class())

  def __generate_export_dir_path(self, model_export_path) -> str:
    export_dir = model_export_path
    if not export_dir[-1] == '/':
      export_dir += '/'
    if self.gradient_cam_export_path[0] == '/':
      self.gradient_cam_export_path = self.gradient_cam_export_path[1:]
    export_dir += self.gradient_cam_export_path
    return export_dir

  def save_class_activation_images(self, model_wrapper, sample: ImageSamples, file_start: str = '', colormap_name: str = 'hsv'):
    # Generate export path
    export_dir = self.__generate_export_dir_path(model_wrapper.model_export_path)

    # Create export-directory
    if not os.path.exists(export_dir):
        os.makedirs(export_dir)

    # Grayscale activation map
    heatmap, heatmap_on_image = apply_colormap_on_image(samples.get_original_image(), self.cam, colormap_name)

    # Save images
    file_name = samples.get_file_name()    
    save_image(heatmap, os.path.join(export_dir, file_start + file_name + self.cam_heatmap_file_desc + self.save_file_type))
    save_image(heatmap_on_image, os.path.join(export_dir, file_start + file_name + self.cam_on_image_heatmap_file_desc + self.save_file_type))
    save_image(self.cam, os.path.join(export_dir, file_start + file_name + self.cam_grayscale_file_desc + self.save_file_type))
    #print(f'Saved GradientCam Image \'{file_start+file_name}_*\' to \'{export_dir}\' ...')

  def show_class_activation_images(self, model_wrapper, sample: ImageSamples, images_per_row: int = 3, figsize: (int, int) = (200, 200)):
    # Generate export path
    export_dir = self.__generate_export_dir_path(model_wrapper.model_export_path)
    file_names = []
    samples.reset_index()
    while samples.next():
      file_names.append(samples.get_file_name())
  
    image_paths = []
    for file_name in file_names:
      paths = glob(f'{export_dir}/*{file_name}*{self.save_file_type}')
      paths.sort()
      image_paths.append(paths)
    
    def __translate_generated_file_name_to_subtitle(path: str) -> str:
      if self.cam_heatmap_file_desc in path:
        return 'CAM (Heatmap)'
      elif self.cam_on_image_heatmap_file_desc in path:
        return 'CAM (Heatmap) on Image'
      elif self.cam_grayscale_file_desc in path:
        return 'CAM (Grayscale)'
      else:
        return ''

    for i, paths in enumerate(image_paths):
      plot_images(paths, images_per_row, f'Image: {file_names[i]}', figsize, __translate_generated_file_name_to_subtitle)



# Neuronal-Networks
---

Here we declare our Neuralnetwork-Wrappers, which is mainly responsible to give access to specified model:

## *VGG-19*

In [ ]:
class Wrapper_VGG19:

  def __init__(self, pretrained: bool = True, model_export_path: str = '/content/results/vgg_19'):
    self.pretrained = pretrained
    self.model_export_path = model_export_path
    self.model = None
  
  def build(self, pretrained: bool = True):
    """
    Will

    """
    self.model = models.vgg19(pretrained=pretrained)
    return self
  
  def summary(self):
    print(f'\n{self.model}')

  def get_targeted_layers(self) -> [int]:
    return [
      0,2,5,7,10,12,14,16,19,21,23,25,28,30,32,34
    ]

# GradientCam

---

To view the result's of the Gradient am, we start with initializing the Samples-Objekt and the GradientCam-Objekt:

In [ ]:
samples = default_image_samples()
gradcam = GradientCam()

## *VGG-19*

Next up we will initialize the neural Networks and run the GradientCam with them.

In [ ]:
vgg19 = Wrapper_VGG19(pretrained=True).build()
vgg19.summary()

In [ ]:
# show displaybar
display_progress_bar(0, len(vgg19.get_targeted_layers()) * len(samples))

for targeted_layer in vgg19.get_targeted_layers():
  # reset samples
  samples.reset_index()
  # loop all samples:
  while samples.next():
    # calculate
    gradcam.run(vgg19, samples, targeted_layer)
    gradcam.save_class_activation_images(vgg19, samples, file_start=f'{targeted_layer:02d}_', colormap_name='hsv')
    # update progress
    update_progress_bar()

In [ ]:
gradcam.show_class_activation_images(vgg19, samples, figsize=(20, 100))